<a href="https://colab.research.google.com/github/eunji1002/pytorch_example/blob/main/example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/datasets/archive.zip -d /content/drive/MyDrive/datasets/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/datasets/train/rings/022.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/023.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/024.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/025.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/026.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/027.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/028.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/029.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/030.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/031.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/032.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/033.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/034.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/035.jpg  
  inflating: /content/driv

In [ ]:
from torch import nn

class SuperLightMobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SuperLightMobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )
        self.num_classes = num_classes
        self.model = nn.Sequential(
            conv_bn(  3,  16, 2),
            conv_dw( 16,  32, 1),
            conv_dw( 32, 64, 2),
            conv_dw(64, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.gap(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

In [10]:
import torch

#hyp parameters
dataset_path = "/content/drive/MyDrive/datasets"
model_weight_save_path = "/content/drive/MyDrive/Models"
num_classes = 100

batch_size = 40
num_workers = 8
lr = 1e-3

total_epoch = 10

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [11]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import os


# Data loading code
traindir = os.path.join(dataset_path, 'train')
testdir = os.path.join(dataset_path, 'test')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)

In [4]:
model = SuperLightMobileNet(num_classes).to(device)

In [5]:
CEloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [12]:
import numpy as np

total_iteration_per_epoch = int(np.ceil(len(train_dataset)/batch_size))

for epoch in range(1, total_epoch + 1):
    model.train()
    for itereation, (input, target) in enumerate(train_loader):
        images = input.to(device)
        labels = target.to(device)

        # Forward pass
        outputs = model(images)
        loss = CEloss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch [{}/{}], Iteration [{}/{}] Loss: {:.4f}'.format(epoch, total_epoch, itereation+1, total_iteration_per_epoch, loss.item()))
    if epoch % 10 == 0:
      torch.save(model.state_dict(), model_weight_save_path + 'model_' + str(epoch) + ".pth")

    model.eval()
    with torch.no_grad():
      correct = 0
      total = 0
      for input, target in test_loader:
          images = input.to(device)
          labels = target.to(device)

          # Forward pass
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += len(labels)
          correct += (predicted == labels).sum().item()

      print('Epoch [{}/{}], Test Accuracy of the model on the {} test images: {} %'.format(epoch, total_epoch, total, 100 * correct / total))

Epoch [1/10], Iteration [1/338] Loss: 4.6468
Epoch [1/10], Iteration [2/338] Loss: 4.7413
Epoch [1/10], Iteration [3/338] Loss: 4.6871
Epoch [1/10], Iteration [4/338] Loss: 4.6770
Epoch [1/10], Iteration [5/338] Loss: 4.7499
Epoch [1/10], Iteration [6/338] Loss: 4.4188
Epoch [1/10], Iteration [7/338] Loss: 4.6744
Epoch [1/10], Iteration [8/338] Loss: 4.6172
Epoch [1/10], Iteration [9/338] Loss: 4.7531
Epoch [1/10], Iteration [10/338] Loss: 4.7246
Epoch [1/10], Iteration [11/338] Loss: 4.4525
Epoch [1/10], Iteration [12/338] Loss: 4.8388
Epoch [1/10], Iteration [13/338] Loss: 4.6056
Epoch [1/10], Iteration [14/338] Loss: 4.7311
Epoch [1/10], Iteration [15/338] Loss: 4.4097
Epoch [1/10], Iteration [16/338] Loss: 4.8127
Epoch [1/10], Iteration [17/338] Loss: 4.6298
Epoch [1/10], Iteration [18/338] Loss: 4.9632
Epoch [1/10], Iteration [19/338] Loss: 4.5134
Epoch [1/10], Iteration [20/338] Loss: 4.9539
Epoch [1/10], Iteration [21/338] Loss: 4.5494
Epoch [1/10], Iteration [22/338] Loss: 4.58